## Nofar Mahrabi 315153726

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime as dt
import datetime 
import csv
import os
from IPython.display import display

In [8]:
# בדיקת הקישור ושליפת התוכן
try:
    url = "https://www.ad.co.il/car?sp261=13909&rp264=2013,2022"
    response = requests.get(url)
    response.raise_for_status()  # בדיקת שגיאות בקריאת הבקשה
except Exception as e:
    print("Error:", e)
else:
    print("Request successful")

  

Request successful


In [9]:
def ads(url):
    try:
        response = requests.get(url, timeout=10)  # הגדלת זמן ההמתנה ל-10 שניות
        response.raise_for_status()  # בדיקת שגיאות בקריאת הבקשה
        soup = BeautifulSoup(response.content, 'html.parser')
        ads = soup.find_all('div', class_='card-body p-md-3')
        ad_links = []
        for ad in ads:
            link = ad.find('a')['href']
            ad_links.append(link)
        return ad_links[:-3]  # Exclude the last three ad links
    except requests.exceptions.Timeout:
        print("The request timed out")
        return []
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return []

In [10]:
def all_ads(base_url):
    all_ad_links = []
    page_num = 1
    while True:
        url = f"{base_url}&pageindex={page_num}"
        ad_links = ads(url)
        if not ad_links:
            break
        all_ad_links.extend(ad_links)
        page_num += 1
    return all_ad_links

In [11]:
def parse_ad_table(tables):
    data_dict = {}
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all(['th', 'td'])
            if len(columns) == 2:
                key = columns[0].text.strip()
                value = columns[1].text.strip()
                data_dict[key] = value
    return data_dict

In [12]:
def extract_dates(soup):
    dates_dict = {}
    creation_date = soup[0]
    last_bump = soup[1]

    date_pattern = r'\d{2}/\d{2}/\d{4}'

    dates_dict['תאריך יצירה'] = re.findall(date_pattern, creation_date.get_text())[0]
    dates_dict['תאריך הקפצה'] = re.findall(date_pattern, last_bump.get_text())[0]
  
    return dates_dict

In [13]:
def get_value_or_nan(description):
    if description:
        value = description.get_text(strip=True)
        if value:
            return value
    return None    


In [18]:
def convert_to_datetime(date_string, date_format="%d/%m/%Y"):
    if isinstance(date_string, datetime):
        return date_string
    try:
        date_obj = dt.strptime(date_string, date_format)
        return date_obj
    except (ValueError, TypeError):
        return None


In [19]:

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

base_url = "https://www.ad.co.il/car?sp261=13909&rp264=2013,2022"
all_ad_links = all_ads(base_url)
print('Here is your data frame:')


ad_data = []
for ad_link in all_ad_links:
    ad_url = f"https://www.ad.co.il{ad_link}"
    try:
        ad_response = requests.get(ad_url, timeout=10)  # הגדלת זמן ההמתנה ל-10 שניות
        ad_response.raise_for_status()  # בדיקת שגיאות בקריאת הבקשה
        if ad_response.status_code == 200:
            ad_result = BeautifulSoup(ad_response.content, 'html.parser')
            ad_soup = ad_result.find('div', class_='tab-content bg-white rounded-3 border p-3 col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
            title = ad_soup.find('h2', class_='card-title').get_text()
            manufacturer = title.split()
            
            table = ad_soup.find_all('table', class_="table table-sm mb-4")
            data_table = parse_ad_table(table)
            
            price = ad_soup.find_all('h2', class_='card-title')[1].text.strip() if len(ad_soup.find_all('h2', class_='card-title')) > 1 else 'NaN'
            
            ad_bottom = ad_soup.find('div', class_='order-first order-sm-0')
            ad_description = ad_bottom.find('p', class_='text-word-break')
            description = get_value_or_nan(ad_description)
            
            dates_soup = ad_bottom.find('div', class_='d-flex flex-row align-items-center justify-content-center flex-wrap')
            date_element = dates_soup.find_all('div', class_='px-3')
            date_dict = extract_dates(date_element)
            
            image_soup = ad_soup.find('div', class_='col-xxl-8 col-lg-8 col-md-7')
            all_images = image_soup.find_all("img")
            main_image = ad_soup.select_one('img.img-fluid')
            total_images_count = len(all_images) - (1 if main_image else 0)
            
            ad_data.append({
                'יצרן': manufacturer[0],
                'דגם': manufacturer[1],
                'מחיר': price,
                **data_table,  
                **date_dict,
                'תיאור': description,
                'מספר תמונות': total_images_count
            })
    except requests.exceptions.Timeout:
        print(f"The request for {ad_url} timed out")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred with {ad_url}:", e)

df = pd.DataFrame(ad_data)
new_order = ['יצרן', 'שנה', 'דגם', 'יד', 'ת. הילוכים', 'נפח', 'סוג מנוע', 'בעלות קודמת', 'בעלות נוכחית', 'אזור', 'עיר', 'מחיר','מספר תמונות', 'תאריך יצירה', 'תאריך הקפצה', 'תיאור',  'צבע', 'ק"מ','טסט עד',]
df = df[new_order]

# הצגה של הדאטה בצורה מסודרת
display(df)

Here is your data frame:


,יצרן,שנה,דגם,יד,ת. הילוכים,נפח,סוג מנוע,בעלות קודמת,בעלות נוכחית,אזור,עיר,מחיר,מספר תמונות,תאריך יצירה,תאריך הקפצה,תיאור,צבע,"ק""מ",טסט עד
0,טויוטה,2021,קורולה,1,אוטומטית,"1,600",בנזין,פרטית,פרטית,רמת גן - גבעתיים,רמת גן,"139,000 ₪",10,10/06/2024,12/06/2024,"בהשדמנות אחת בדור !!!!!!! רכב שמור,קילומטראז נ...",לבן פנינה,"14,000",NaN
1,טויוטה,2013,פריוס,3,אוטומטית,"1,800",היבריד,פרטית,פרטית,בית שמש והסביבה,בית שמש,"44,000 ₪",9,30/05/2024,05/06/2024,"רכב במצב מעולה שמור מאוד. ללא תאונות, סוללה הי...",לבן,"201,000",10/2024
2,טויוטה,2024,יאריס,1,אוטומטית,"1,500",בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,150 ₪,6,08/04/2024,22/05/2024,סט פנסים ENERGIZER לשעת חירום.\nפנס ראש+ פנס י...,לבן,NaN,NaN
3,טויוטה,2016,יאריס,2,אוטומטית,"1,300",בנזין,פרטית,פרטית,הוד השרון והסביבה,הוד השרון,"61,000 ₪",3,22/05/2024,22/05/2024,"יאריס שנת 2016 יד 01, דגם SKY כולל גג פנורמי,מ...",לבן,"75,000",01/2025
4,טויוטה,2021,אוולון,2,אוטומטית,"2,500",בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,1 ₪,4,16/05/2024,16/05/2024,None,לבן,"190,000",09/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,טויוטה,2014,קורולה,1,אוטומטית,"1,600",בנזין,NaN,NaN,חולון - בת ים,חולון,"60,000 ₪",0,28/04/2022,28/04/2022,טויוטה קורולה G.L &lt;br/&gt;&lt;br/&gt;פירוט:...,NaN,"131,000",NaN
226,טויוטה,2013,ורסו,3,אוטומטית,"1,800",בנזין,פרטית,פרטית,מושבים במרכז,אחיעזר,"49,000 ₪",0,22/12/2021,22/12/2021,רכב מצויין שמור מאוד מטופל כהלכה,אפור עכבר,"194,000",NaN
227,טויוטה,2013,ורסו,4,אוטומטית,"1,800",בנזין,ליסינג,פרטית,מושבים במרכז,אחיעזר,"56,000 ₪",0,15/11/2021,15/11/2021,רכב יפה מבפנים ומבחוץ מצב מכני מעולה מטופל מרא...,אפור עכבר,"192,000",NaN
228,טויוטה,2015,פריוס,2,אוטומטית,"1,800",היבריד,ליסינג,פרטית,ירושלים והסביבה,ירושלים,"85,000 ₪",0,12/10/2021,12/10/2021,None,כסוף,"125,000",NaN


In [24]:
df = df.rename(columns={'יצרן': 'manufactor'})
df['manufactor'] = df['manufactor'].astype(str)
df = df.rename(columns={'שנה': 'Year'})
df['Year'] = df['Year'].astype(int)
df = df.rename(columns={'דגם': 'model'})
df['model'] = df['model'].astype(str)
df = df.rename(columns={'יד': 'Hand'})
df['Hand'] = pd.to_numeric(df['Hand'], errors='coerce').astype('Int64')
df = df.rename(columns={'ת. הילוכים': 'Gear'})
df['Gear'] = pd.Categorical(df['Gear'])
df = df.rename(columns={'נפח': 'Engine_capacity'})
df['Engine_capacity'] = df['Engine_capacity'].astype(str).str.replace(',', '')  # המרת העמודה לטיפוס מחרוזת ואז החלפת הפסיקים בריק
df['Engine_capacity'] = df['Engine_capacity'].astype(int)  # המרת העמודה לטיפוס int

df = df.rename(columns={'סוג מנוע': 'Engine_type'})
df['Engine_type'] = pd.Categorical(df['Engine_type'])
df = df.rename(columns={'בעלות קודמת': 'Prev_ownership'})
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'])
df = df.rename(columns={'בעלות נוכחית': 'Curr_ownership'})
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'])
df = df.rename(columns={'אזור': 'Area'})
df['Area'] = df['Area'].astype(str, errors='ignore')
df = df.rename(columns={'עיר': 'City'})
df['City'] = df['City'].astype(str)
df = df.rename(columns={'מחיר': 'Price'})
df['Price'] = df['Price'].astype(str).str.replace('[₪,]', '', regex=True).astype(float)
df = df.rename(columns={'מספר תמונות': 'Pic_num'})
df['Pic_num'] = df['Pic_num'].astype(int)
df = df.rename(columns={'תאריך יצירה': 'Cre_date'})
df['Cre_date'] = pd.to_datetime(df['Cre_date'], format='%d/%m/%Y', errors='coerce')
df = df.rename(columns={'תאריך הקפצה': 'Repub_date'})
df['Repub_date'] = pd.to_datetime(df['Repub_date'], format='%d/%m/%Y', errors='coerce')
df = df.rename(columns={'תיאור': 'Description'})
df['Description'] = df['Description'].astype(str)
df = df.rename(columns={'צבע': 'Color'})
df['Color'] = df['Color'].astype(str)
df = df.rename(columns={'ק"מ': 'Km'})
df['Km'] = pd.to_numeric(df['Km'].str.replace(',', ''), errors='coerce').astype('Int64')
df = df.rename(columns={'טסט עד': 'Test'})
# המרת הנתונים לפורמט תאריך
df['Test'] = pd.to_datetime(df['Test'], format='%m/%Y', errors='coerce')
df['Test'] = (df['Test'] - pd.Timestamp.today()).dt.days.astype('Int64')



print(df.dtypes)
display(df)





manufactor                 object
Year                        int32
model                      object
Hand                        Int64
Gear                     category
Engine_capacity             int32
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     int32
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          Int64
Test                        Int64
dtype: object


,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,טויוטה,2021,קורולה,1,אוטומטית,1600,בנזין,פרטית,פרטית,רמת גן - גבעתיים,רמת גן,139000.0,10,2024-06-10,2024-06-12,"בהשדמנות אחת בדור !!!!!!! רכב שמור,קילומטראז נ...",לבן פנינה,14000,<NA>
1,טויוטה,2013,פריוס,3,אוטומטית,1800,היבריד,פרטית,פרטית,בית שמש והסביבה,בית שמש,44000.0,9,2024-05-30,2024-06-05,"רכב במצב מעולה שמור מאוד. ללא תאונות, סוללה הי...",לבן,201000,109
2,טויוטה,2024,יאריס,1,אוטומטית,1500,בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,150.0,6,2024-04-08,2024-05-22,סט פנסים ENERGIZER לשעת חירום.\nפנס ראש+ פנס י...,לבן,<NA>,<NA>
3,טויוטה,2016,יאריס,2,אוטומטית,1300,בנזין,פרטית,פרטית,הוד השרון והסביבה,הוד השרון,61000.0,3,2024-05-22,2024-05-22,"יאריס שנת 2016 יד 01, דגם SKY כולל גג פנורמי,מ...",לבן,75000,201
4,טויוטה,2021,אוולון,2,אוטומטית,2500,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,1.0,4,2024-05-16,2024-05-16,None,לבן,190000,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,טויוטה,2014,קורולה,1,אוטומטית,1600,בנזין,NaN,NaN,חולון - בת ים,חולון,60000.0,0,2022-04-28,2022-04-28,טויוטה קורולה G.L &lt;br/&gt;&lt;br/&gt;פירוט:...,nan,131000,<NA>
226,טויוטה,2013,ורסו,3,אוטומטית,1800,בנזין,פרטית,פרטית,מושבים במרכז,אחיעזר,49000.0,0,2021-12-22,2021-12-22,רכב מצויין שמור מאוד מטופל כהלכה,אפור עכבר,194000,<NA>
227,טויוטה,2013,ורסו,4,אוטומטית,1800,בנזין,ליסינג,פרטית,מושבים במרכז,אחיעזר,56000.0,0,2021-11-15,2021-11-15,רכב יפה מבפנים ומבחוץ מצב מכני מעולה מטופל מרא...,אפור עכבר,192000,<NA>
228,טויוטה,2015,פריוס,2,אוטומטית,1800,היבריד,ליסינג,פרטית,ירושלים והסביבה,ירושלים,85000.0,0,2021-10-12,2021-10-12,None,כסוף,125000,<NA>


In [22]:
column_dtype = df['Cre_date'].dtype
print(column_dtype)

object


In [31]:
# נתיב יחסי לקובץ שבו נשמור את הנתונים
file_path = "C:\\Users\\נופר מחרבי\\pythonNotebook\\MyProject\\output.csv"
df.to_csv(file_path, index=False, encoding='utf-8-sig')
# הודעה על שמירת הקובץ בהצלחה
print(f"The file has been saved successfully at {file_path}")

The file has been saved successfully at C:\Users\נופר מחרבי\pythonNotebook\MyProject\output.csv
